### **Préparation et équilibrage des données**

In [1]:
# Importation des bibliothèques
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from collections import Counter

# Chargement des données
data = pd.read_csv('../../../data/creditcard.csv')
# Vérification des dimensions et aperçu des données
print(f"Dimensions des données : {data.shape}")
data.head()


Dimensions des données : (284807, 31)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [2]:
# Vérification des valeurs manquantes
missing_values = data.isnull().sum()
print("Valeurs manquantes par colonne :\n", missing_values[missing_values > 0])

Valeurs manquantes par colonne :
 Series([], dtype: int64)


Aucune valeur manquante n'est détecté donc on peut continuer sans traitement supplémentaire .

**Normalisation des variables Time et Amount**

Ces deux variables ne sont pas transformées par ACP, il est donc important de les normaliser.

In [3]:
# Standardisation des colonnes 'Time' et 'Amount'
scaler = StandardScaler()
data[['Time', 'Amount']] = scaler.fit_transform(data[['Time', 'Amount']])

# Vérification des données normalisées
data[['Time', 'Amount']].describe()


,Time,Amount
count,2.848070e+05,2.848070e+05
mean,-3.065637e-16,2.913952e-17
std,1.000002e+00,1.000002e+00
min,-1.996583e+00,-3.532294e-01
25%,-8.552120e-01,-3.308401e-01
50%,-2.131453e-01,-2.652715e-01
75%,9.372174e-01,-4.471707e-02
max,1.642058e+00,1.023622e+02


La standardisation consiste à transformer les données pour qu'elles aient une moyenne de 0 et un écart-type de 1, ce qui est utile pour rendre les caractéristiques comparables .

**Traitement du déséquilibre des classes (SMOTE)**

Le jeu de données est fortement déséquilibré : il y a beaucoup plus de transactions légitimes (Class = 0) que frauduleuses (Class = 1). SMOTE (Synthetic Minority Oversampling Technique) est utilisé pour équilibrer les classes.

In [ ]:
from collections import Counter
import pandas as pd

# Séparation des caractéristiques et de la cible
X = data.drop('Class', axis=1)
y = data['Class']

# Vérification des proportions des classes avant équilibrage
print(f"Répartition des classes avant équilibrage : {Counter(y)}")

# Application de SMOTE pour équilibrer les classes
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Vérification des proportions des classes après équilibrage
print(f"Répartition des classes après équilibrage : {Counter(y_resampled)}")

# Conversion en DataFrame pour sauvegarde
balanced_data = pd.DataFrame(X_resampled, columns=X.columns)
balanced_data['Class'] = y_resampled  # Ajout de la colonne cible

# Chemin pour sauvegarder le jeu de données
output_path = '../data/balanced_data.csv'

# Sauvegarde du jeu de données équilibré
# balanced_data.to_csv(output_path, index=False)

print(f"Jeu de données équilibré sauvegardé dans : {output_path}")


Répartition des classes avant équilibrage : Counter({0: 284315, 1: 492})
Répartition des classes après équilibrage : Counter({0: 284315, 1: 284315})
Jeu de données équilibré sauvegardé dans : ../data/balanced_data.csv


Avant l'application de la méthode **SMOTE (Synthetic Minority Over-sampling Technique)**, les données présentaient un déséquilibre marqué, avec une forte majorité appartenant à la classe 0 (non-frauduleuses). Cette situation pouvait entraîner un biais des modèles d'apprentissage automatique, les rendant inefficaces pour détecter les transactions frauduleuses, qui sont largement sous-représentées.

Après l'application de **SMOTE**, les classes ont été équilibrées grâce à la génération synthétique d'exemples pour la classe minoritaire (frauduleuse). Cet équilibrage améliore significativement les performances des modèles, leur permettant de mieux distinguer les transactions frauduleuses des non-frauduleuses et d'éviter un apprentissage biaisé en faveur de la classe dominante.

**Division des données en ensembles d'entraînement et de test**

In [6]:
# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Vérification des dimensions
print(f"Taille de l'ensemble d'entraînement : {X_train.shape}")
print(f"Taille de l'ensemble de test : {X_test.shape}")


Taille de l'ensemble d'entraînement : (454904, 30)
Taille de l'ensemble de test : (113726, 30)


In [8]:
# Enregistrer les ensembles d'entraînement et de test pour une utilisation future.
X_train.to_csv('../data/X_train.csv', index=False)
X_test.to_csv('../data/X_test.csv', index=False)
y_train.to_csv('../data/y_train.csv', index=False)
y_test.to_csv('../data/y_test.csv', index=False)

print("Données prétraitées et enregistrées avec succès.")


Données prétraitées et enregistrées avec succès.


#### **Étapes du prétraitement**
1. Gestion des valeurs manquantes : aucune valeur manquante détectée.
2. Normalisation : `Time` et `Amount` standardisés pour alignement avec les caractéristiques transformées par ACP.
3. Équilibrage des classes : utilisation de SMOTE pour équilibrer la répartition des transactions frauduleuses et non-frauduleuses.
4. Division des données : ensemble d'entraînement et de test divisés de manière équilibrée, prêts pour l'entraînement des modèles.
